In [1]:
import pandas as pd

# Load the CSV file
csv_file_path = 'new_test.csv'
df = pd.read_csv(csv_file_path)

# Define the BKT parameters
BKTparams = {
    "p_transit": 0.2,
    "p_slip": 0.1,
    "p_guess": 0.2,
    "p_know": 0.25  # Initial knowledge probability
}

# Function to update p_know using BKT
def update_p_know(p_know_tminus1, outcome, BKTparams):
    p_slip = BKTparams["p_slip"]
    p_guess = BKTparams["p_guess"]
    p_transit = BKTparams["p_transit"]

    if outcome.lower() == "correct":
        p_know_given_obs = (p_know_tminus1 * (1 - p_slip)) / ((p_know_tminus1 * (1 - p_slip)) + ((1 - p_know_tminus1) * p_guess))
    else:
        p_know_given_obs = (p_know_tminus1 * p_slip) / ((p_know_tminus1 * p_slip) + ((1 - p_know_tminus1) * (1 - p_guess)))

    # Update p_know and apply the transition
    p_know = p_know_given_obs + (1 - p_know_given_obs) * p_transit
    return round(p_know, 2)  # Round to 2 decimal places

# Define function to calculate "Critical Struggle" based on the criteria provided
def calculate_struggle_status(df):
    struggle_status = []
    
    # Define thresholds for struggle (based on previous data and conditions)
    error_threshold = 6  # error count threshold, can adjust as needed
    attempt_threshold = 10  # attempt threshold, can adjust as needed
    mastery_threshold = 0.8  # mastery level threshold, can adjust as needed
    
    onboardSkills = {}  # Store the student's skill knowledge levels

    for i in range(len(df)):
        prev_p_know = df['CF (Skill Previous p-Known)'].iloc[i]
        new_p_know = df['CF (Skill New p-Known)'].iloc[i]
        attempts = df['Attempt At Step'].iloc[i]
        outcome = df['Outcome'].iloc[i]  #'Outcome' column has correct/incorrect actions
        
        # Check if this skill has been encountered previously
        skill_name = df['Problem Name'].iloc[i]  # Example column to identify skills
        if skill_name not in onboardSkills:
            onboardSkills[skill_name] = BKTparams["p_know"]  # Initialize with the default p_know

        # Update p_know based on BKT
        updated_p_know = update_p_know(onboardSkills[skill_name], outcome, BKTparams)
        onboardSkills[skill_name] = updated_p_know  # Update the skill's p_know

        # If proficiency is below threshold and attempts are high, consider it as a struggle
        if (updated_p_know < mastery_threshold or new_p_know < mastery_threshold) and attempts >= attempt_threshold:
            struggle_status.append(True)
        else:
            struggle_status.append(False)
    
    return struggle_status

# Apply the function to calculate struggle status
df['Critical Struggle'] = calculate_struggle_status(df)

# Save the modified dataframe with the new 'Critical Struggle' column to a new CSV file
output_file_path = 'new_test-critical-struggle.csv'
df.to_csv(output_file_path, index=False)

output_file_path  # Return the path to the new file


'new_test-critical-struggle.csv'